In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime as dt
import os

In [3]:
os.chdir("../")
df_tx = pd.read_csv('data/15minute_data_austin.csv')
df_ca = pd.read_csv('data/15minute_data_california.csv')
df_ny = pd.read_csv('data/15minute_data_newyork.csv')
df_ca.shape

(805524, 79)

In [4]:
def clean_up_dataset(dataframe):
    sorted_df = dataframe.sort_values(by= ['local_15min'])
    all_sorted_time = sorted_df['local_15min'].unique()
    time_ids = {}
    unique_ids = {}
    counter = 0
    dataframe = dataframe.fillna(0)
    
    # Assign a unique numerical id to each timestamp
    for i in all_sorted_time:
        time_ids.update({i: counter})
        unique_ids.update({counter: i})
        counter += 1

    
    def get_front_back(df, date):
        if time_ids[date] - 1 > 0:
            back = unique_ids[time_ids[date] - 1]
        else:
            back = date
        if time_ids[date] + 1 < len(unique_ids):
            front = unique_ids[time_ids[date] + 1]
        else:
            front = date
        return (df[df['local_15min'] == front]['grid'], df[df['local_15min'] == back]['grid'])
    
    def missing_times(lst1, lst2):
        # takes unique list and finds the difference in other list
        return list(set(lst1).difference(lst2))
    
    # Append missing data values
    time_labels = []
    grid_interpret = []
    data_labels =[]
    num_uid = 0
    for k in dataframe['dataid'].unique():
        num_uid += 1
        cur_mis = missing_times(all_sorted_time, dataframe[dataframe['dataid'] == k]['local_15min'].unique())
        print(f"Household ID {k} is missing {len(cur_mis)} timestamps.")
        for j in cur_mis:
            time_labels += [j]
            grid_interpret += [np.nan]
            data_labels += [k]
            
    sorted_id = dataframe.append(pd.DataFrame({'local_15min': time_labels, 'grid': grid_interpret, 'dataid': data_labels}),
                    ignore_index = True)
    
    # Resort by data labels
    sorted_id = sorted_id.sort_values(by= ['dataid'])
    sorted_id.reset_index(drop=True,inplace=True)

    # Apply interpolation by indexing into separate UIDs
    df_list = []
    for uid in sorted_id['dataid'].unique():
        grid_pts = sorted_id.loc[sorted_id['dataid']==uid,:].copy()
        grid_pts = grid_pts.sort_values(by=['local_15min'],ignore_index=True)
        interp = grid_pts.interpolate(method='polynomial',order = 5)
        df_list.append(interp)
      
    sorted_id = pd.concat(df_list,ignore_index=True)
    return sorted_id

In [111]:
sorted_cleaned_tx = clean_up_dataset(df_tx)
# sorted_cleaned_ca = clean_up_dataset(df_ca)
sorted_cleaned_ny = clean_up_dataset(df_ny)

Household ID 661 is missing 4 timestamps.
Household ID 1642 is missing 388 timestamps.
Household ID 2335 is missing 568 timestamps.
Household ID 2361 is missing 52 timestamps.
Household ID 2818 is missing 56 timestamps.
Household ID 3039 is missing 0 timestamps.
Household ID 3456 is missing 104 timestamps.
Household ID 3538 is missing 0 timestamps.
Household ID 4031 is missing 0 timestamps.
Household ID 4373 is missing 504 timestamps.
Household ID 4767 is missing 77 timestamps.
Household ID 5746 is missing 300 timestamps.
Household ID 6139 is missing 0 timestamps.
Household ID 7536 is missing 52 timestamps.
Household ID 7719 is missing 156 timestamps.
Household ID 7800 is missing 16 timestamps.
Household ID 7901 is missing 144 timestamps.
Household ID 7951 is missing 0 timestamps.
Household ID 8156 is missing 52 timestamps.
Household ID 8386 is missing 0 timestamps.
Household ID 8565 is missing 0 timestamps.
Household ID 9019 is missing 108 timestamps.
Household ID 9160 is missing 32 t

In [113]:
def get_date(date):
    if type(date) == str:
        dtobj = dt.datetime.strptime(date, '%Y-%m-%d %H:%M:%S-%f')
        return dtobj.year * 10000 + dtobj.month * 100 + dtobj.day
    return date.year * 10000 + date.month * 100 + date.day

def all_days(df):
    days = []
    i = 96
    while i < len(df):
        days.append(get_date(df.iloc[i-1]['local_15min']))
    return days

In [114]:
def multi_index_dataframe(df):
    df_days_all = df[df['dataid'] == df['dataid'][0]]['local_15min']
    df_days = []
    i = 96
    while i < len(df_days_all):
        df_days += [get_date(df_days_all[i-1])]
        i += 96
    iterables = [df['dataid'].unique(), df_days]
    multi_index = pd.MultiIndex.from_product(iterables, names=["dataid", "date"])    
    vals = []
    for houseid in df['dataid'].unique():
        temp = df[df['dataid'] == houseid]
        temp_dict = {}
        i = 96
        while i < len(temp):
            vals_temp = list(temp[i - 96:i]['grid'])
            vals += [vals_temp]
            i += 96
    df_final = pd.DataFrame(vals, index = multi_index, columns = np.arange(96))
    return df_final

In [115]:
multi_index_dataframe(sorted_cleaned_tx)

0      1      2      3      4      5      6      7   \
dataid date                                                               
661    20180101  0.000  0.000  0.000  0.000  1.447  1.633  1.410  1.696   
       20180102  1.120  0.897  0.892  1.059  0.940  0.640  0.708  0.803   
       20180103  0.700  1.202  0.632  0.955  0.450  0.778  0.718  0.656   
       20180104  0.826  0.963  0.623  0.914  0.716  0.694  0.624  0.469   
       20180105  0.794  0.977  0.688  0.769  0.628  0.752  0.393  0.728   
...                ...    ...    ...    ...    ...    ...    ...    ...   
9922   20181227  1.101  1.098  0.963  0.963  1.096  1.034  1.066  1.083   
       20181228  0.997  0.960  0.922  0.913  1.444  1.054  1.134  1.019   
       20181229  1.310  1.077  0.926  1.453  0.858  1.352  1.006  1.021   
       20181230  1.448  1.132  1.195  1.255  0.937  1.230  1.036  1.563   
       20181231  0.957  1.448  0.868  1.340  1.225  0.942  1.408  0.837   

                    8      9   ...     86     87     88     89     90     91  \
dataid date                    ...                                             
661    20180101  1.095  1.251  ...  1.690  1.752  1.391  1.541  1.471  1.456   
       20180102  0.413  0.817  ...  1.449  1.558  1.184  0.841  1.063  0.853   
       20180103  0.906  0.617  ...  1.457  1.612  1.416  1.507  1.246  1.547   
       20180104  0.763  0.614  ...  1.076  1.305  1.276  1.329  1.062  1.103   
       20180105  0.459  0.786  ...  1.217  1.031  0.919  0.657  0.845  0.999   
...                ...    ...  ...    ...    ...    ...    ...    ...    ...   
9922   20181227  0.968  0.975  ...  0.994  0.991  0.987  1.447  0.938  0.981   
       20181228  1.017  1.177  ...  1.651  1.169  1.858  1.379  1.364  1.320   
       20181229  1.534  0.982  ...  1.777  2.376  1.320  1.190  1.405  1.007   
       20181230  0.983  1.570  ...  2.476  1.998  2.216  2.115  1.057  1.449   
       20181231  1.504  1.052  ...  2.334  2.704  2.238  2.724  2.367  1.364   

                    92     93     94     95  
dataid date                                  
661    20180101  1.264  1.025  0.654  0.873  
       20180102  0.720  1.020  1.073  0.738  
       20180103  0.932  0.605  0.823  0.802  
       20180104  0.901  0.867  0.745  0.865  
       20180105  0.651  0.586  0.813  0.715  
...                ...    ...    ...    ...  
9922   20181227  1.032  1.018  1.038  0.998  
       20181228  1.273  0.885  1.402  0.878  
       20181229  1.543  0.816  1.428  0.870  
       20181230  0.902  0.986  1.542  0.965  
       20181231  1.580  1.073  1.346  1.316  

[9100 rows x 96 columns]

In [116]:
multi_index_dataframe(sorted_cleaned_ny)

0      1      2      3      4      5      6      7   \
dataid date                                                               
27     20190501  0.997  0.750  0.608  1.057  0.518  1.163  0.591  1.155   
       20190502  0.300  0.275  0.296  0.836  0.248  0.269  0.753  0.374   
       20190503  0.352  0.355  0.265  0.219  0.304  0.342  0.336  0.301   
       20190504  0.408  0.311  0.359  0.364  0.266  0.209  0.303  0.413   
       20190505  0.544  0.503  0.781  0.368  0.901  0.369  0.658  0.600   
...                ...    ...    ...    ...    ...    ...    ...    ...   
9053   20191026  0.695  0.710  0.712  1.175  0.805  0.728  0.734  0.728   
       20191027  0.673  0.688  0.695  0.635  0.673  0.690  0.692  0.630   
       20191028  1.104  1.172  0.955  0.681  0.545  0.516  0.516  0.516   
       20191029  0.798  0.801  0.784  0.735  0.376  0.304  0.258  0.227   
       20191030  0.752  0.707  0.621  0.590  0.596  0.602  0.684  0.691   

                    8      9   ...     86     87     88     89     90     91  \
dataid date                    ...                                             
27     20190501  0.840  0.697  ...  0.557  1.052  0.561  0.596  0.284  0.394   
       20190502  0.249  0.374  ...  0.741  0.830  0.749  0.703  0.224  0.233   
       20190503  0.304  0.304  ...  0.666  0.477  0.331  0.331  0.316  0.316   
       20190504  0.290  0.209  ...  1.449  0.782  2.082  1.291  1.140  0.811   
       20190505  0.440  0.808  ...  1.196  0.677  0.859  0.458  0.599  0.507   
...                ...    ...  ...    ...    ...    ...    ...    ...    ...   
9053   20191026  0.213  0.178  ...  0.407  0.292  0.199  0.155  0.228  0.505   
       20191027  0.636  0.230  ...  1.266  1.065  1.054  1.088  1.187  1.151   
       20191028  0.517  0.581  ...  0.500  0.586  0.795  0.777  0.782  0.715   
       20191029  0.232  0.337  ...  0.619  0.608  0.605  0.592  0.532  0.536   
       20191030  0.680  0.718  ...  0.905  0.931  0.908  0.972  1.071  1.383   

                    92     93     94     95  
dataid date                                  
27     20190501  0.356  0.219  0.605  0.304  
       20190502  0.355  0.409  0.294  0.260  
       20190503  0.231  0.277  0.361  0.376  
       20190504  0.592  1.059  0.969  0.665  
       20190505  0.564  0.669  0.540  0.631  
...                ...    ...    ...    ...  
9053   20191026  0.588  0.713  0.736  0.700  
       20191027  1.126  1.126  1.090  1.096  
       20191028  0.750  0.756  0.863  0.797  
       20191029  0.545  0.594  0.701  0.751  
       20191030  1.440  1.388  1.287  1.297  

[4575 rows x 96 columns]